<a href="https://colab.research.google.com/github/shivendrra/Machine-Learning/blob/main/Sentiment%20analysis/IMDB_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/MyDrive/IMDB Dataset.csv'

import pandas as pd
import numpy as np
data = pd.read_csv(file_path)
print(data.head)

<bound method NDFrame.head of                                                   review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]>


In [3]:
# data preprocessing
data['review'].replace('https?://\S+|www\.\S+'," ",regex=True,inplace=True)
data['review'].replace('<.*?>'," ",regex=True,inplace=True)
data['review'].replace('@\w+'," ",regex=True,inplace=True)
data['review'].replace('#\w+'," ",regex=True,inplace=True)
data['review'].replace("[^\w\s\d]"," ",regex=True,inplace=True)
data['review'].replace(r'( +)'," ",regex=True,inplace=True)
data['review'].replace("[^a-zA-Z]"," ",regex=True,inplace=True)

print(data.head)

<bound method NDFrame.head of                                                   review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production The filming tech...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there s a family where a little boy ...  negative
4      Petter Mattei s Love in the Time of Money is a...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot bad dialogue bad acting idiotic direc...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I m going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]>


In [4]:
# data seperation
reviews = data['review'].values.astype('U')
reviews = np.array(reviews)

response = data['sentiment'].values.astype('U')
response = np.array(response)

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re

lm = WordNetLemmatizer()
ps = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
corpus = []
for i in range(len(reviews)):
  sen = reviews[i]
  token = nltk.sent_tokenize(sen)
  for j in range(len(token)):
    review = token[j]
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

print(corpus[:5])

['one review mention watch oz episod hook right exactli happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side', 'wond

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x_feat = cv.fit_transform(corpus).toarray()
x_feat = pd.DataFrame(x_feat)
x_feat = np.array(x_feat.iloc[:5500])

y_feat = pd.get_dummies(data['sentiment'])
y_feat = y_feat.iloc[:5500]

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_feat, y_feat, test_size=0.15, random_state=0)
y_train = y_train['positive'].values.flatten()

In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

In [10]:
from sklearn.naive_bayes import MultinomialNB

spam_detection = MultinomialNB().fit(x_train, y_train)
y_pred = spam_detection.predict(x_test)
y_test = y_test['positive'].values.flatten()

In [14]:
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test, y_pred)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

print("accuracy is ", accuracy)
print("confusion matrix \n", conf)

accuracy is  0.8351515151515152
confusion matrix 
 [[348  52]
 [ 84 341]]
